In [1]:
import torch
import scipy.io as sio

In [2]:
import pickle
import numpy as np
import tensorflow as tf


The next block is a copy of part of the original code

In [3]:
#Utility functions, provided in the release of the code from the original MeshGraphNets study:
#https://github.com/deepmind/deepmind-research/tree/master/meshgraphnets

def triangles_to_edges_old(faces):
  """Computes mesh edges from triangles.
     Note that this triangles_to_edges method was provided as part of the
     code release for the MeshGraphNets paper by DeepMind, available here:
     https://github.com/deepmind/deepmind-research/tree/master/meshgraphnets
  """
  # collect edges from triangles
  edges = tf.concat([faces[:, 0:2],
                     faces[:, 1:3],
                     tf.stack([faces[:, 2], faces[:, 0]], axis=1)], axis=0)
  # those edges are sometimes duplicated (within the mesh) and sometimes
  # single (at the mesh boundary).
  # sort & pack edges as single tf.int64
  receivers = tf.reduce_min(edges, axis=1)
  senders = tf.reduce_max(edges, axis=1)
  packed_edges = tf.bitcast(tf.stack([senders, receivers], axis=1), tf.int64)
  print(packed_edges)
  #print(tf.unique(packed_edges)[0])
  # remove duplicates and unpack
  unique_edges = packed_edges
  #unique_edges = tf.bitcast(tf.unique(packed_edges)[0], tf.int32)
  senders, receivers = tf.unstack(unique_edges, axis=1)
  # create two-way connectivity
  return (tf.concat([senders, receivers], axis=0),
          tf.concat([receivers, senders], axis=0))




The next block is my rewrite of the function, but replace the tensorflow to numpy

In [4]:
#Utility functions, provided in the release of the code from the original MeshGraphNets study:
#https://github.com/deepmind/deepmind-research/tree/master/meshgraphnets

def triangles_to_edges(faces):
  edges = np.concatenate([faces[:, 0:2],
                     faces[:, 1:3],
                     np.stack([faces[:, 2], faces[:, 0]], axis=1)], axis=0)
  # those edges are sometimes duplicated (within the mesh) and sometimes
  # single (at the mesh boundary).
  # sort & pack edges as single tf.int64
  receivers = np.min(edges, axis=1)
  senders = np.max(edges, axis=1)

  packed_edges = np.stack([senders, receivers], axis=1)
  # remove duplicates and unpack
  unique_edges = np.unique(packed_edges, axis=0)
  senders = unique_edges[:, 0]
  receivers = unique_edges[:, 1]
   # create two-way connectivity
  return (np.concatenate([senders, receivers], axis=0),
         np.concatenate([receivers, senders], axis=0))

The "load_matlab_data" block is used to convert the matlab output "x_all.mat" to pickle file. 

In [5]:
''' This is hard coded for 2 cell systems, will be adjusted '''

def cell_to_vertex(cell, reverse = False):
    if reverse == False:
        key_cell = cell.sort()
        mapper = {"0 1 2": 0, "0 1 3": 1}
        return mapper[" ".join([str(c) for c in key_cell])]
    else:
        rev_mapper = {0: [0, 1, 2], 1: [0, 1, 3]}
        return rev_mapper[cell]
            



def load_matlab_data(file_name: str = "x_all.mat"):
    
    from copy import deepcopy
    x_full = sio.loadmat(file_name)
    
    x_all = x_full['x_all']
    # adj_mat = x_full["adj_mat"]
    
    ''' These are specific for 2 cell systems'''
    dimension = 3
    edgenum = 5 # np.sum(adj_mat)
    cellnum = 2

    nodenum = round(x_all.shape[0]/3) # number of nodes with x,y,z coordinates
    timeall = x_all.shape[1] # number of timesteps
    x_all_expand = np.reshape(x_all,( nodenum, dimension, timeall))
    x_all_expand_reshaped = np.transpose(x_all_expand,[2,0,1] )


    edge_index =range(0, edgenum)

    #edge_node = [[0,1],[0,2],[0,3],[1,2],[1,3]] #np.argwhere(adj_mat == 1)
    cells = np.zeros((timeall, cellnum, 3))

    world_pos = deepcopy(x_all_expand_reshaped)
    mesh_pos = np.zeros_like(world_pos)
    node_type = np.zeros((timeall, nodenum, 1)) # TODO: Check the node type

    for ii in range(0, timeall): 
        for cell in range(cellnum): # 0 or 1
            cells[ii,cell,:] = cell_to_vertex(cell, reverse=True)
        mesh_pos[ii,:,:] =  x_all_expand_reshaped[0,:,:] # TODO: Please CHK

    dataset = [{
        "cells": cells,
        "mesh_pos": mesh_pos,
        "node_type": node_type,
        "world_pos": world_pos,

    }]

    fileinp = "test_dataset.pickle"
    with open(fileinp, "wb") as file:
        pickle.dump(dataset, file, pickle.HIGHEST_PROTOCOL)
    
    return dataset
# with open('test_dataset.pickle', "wb") as file:
#     dataset = pickle.load(file)

In [6]:
dataset = load_matlab_data()
print(np.size(dataset))
print(dataset)


1
[{'cells': array([[[0., 1., 2.],
        [0., 1., 3.]],

       [[0., 1., 2.],
        [0., 1., 3.]],

       [[0., 1., 2.],
        [0., 1., 3.]],

       ...,

       [[0., 1., 2.],
        [0., 1., 3.]],

       [[0., 1., 2.],
        [0., 1., 3.]],

       [[0., 1., 2.],
        [0., 1., 3.]]]), 'mesh_pos': array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
        [ 5.00000000e-01, -8.66024853e-01, -9.77237984e-04]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
        [ 5.00000000e-01, -8.66024853e-01, -9.77237984e-04]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.00000000e-01,  8.66025404e-01,  0.00000000e+00],
        [ 

In [7]:
# import libraries
import pickle
import json
import sys
import os

# open pickle file
with open('test_dataset.pickle', 'rb') as infile:
    obj = pickle.load(infile)

# convert pickle object to json object
json_obj = json.loads(json.dumps(obj, default=str))

# write the json file
with open(
        os.path.splitext('test_dataset.pickle')[0] + '.json',
        'w',
        encoding='utf-8'
    ) as outfile:
    json.dump(json_obj, outfile, ensure_ascii=False, indent=4)

In [8]:
!! The following code is used to convert numpy to pytorch data structure, saved as ".pt" directly. So we can skip the tensorflow file ".tfrecord". You may need to rewrite the data. You need to make some modifications

["'The' is not recognized as an internal or external command,",
 'operable program or batch file.']

In [9]:
from torch_geometric.data import Data
from tqdm import tqdm

data = dataset
trajectory = 0
number_ts = len(data[trajectory]['world_pos'])-1
dt = 0.01
data_list = []
for ts in tqdm(range(len(data[trajectory]['world_pos'])-1)):

    if(ts==number_ts):
        break

    #momentum = torch.tensor(np.array(data[trajectory]['world_pos'][ts]))
    #node_type = torch.tensor(np.array(tf.one_hot(tf.convert_to_tensor(data[trajectory]['node_type'][0]), 9))).squeeze(1)
    #x = torch.cat((momentum,node_type),dim=-1).type(torch.float)
       #node_type = torch.tensor(np.array(np.one_hot(tf.convert_to_tensor(data[trajectory]['node_type'][0]), NodeType.SIZE))).squeeze(1)
    #x = momentum

    #Get edge indices in COO format
    # edges = triangles_to_edges_old(tf.convert_to_tensor(np.array(data[trajectory]['cells'][ts])))
    '''These might not be needed '''
    edges = triangles_to_edges(np.array(data[trajectory]['cells'][ts])) 
    edge_index = torch.cat( (torch.tensor(edges[0]).unsqueeze(0) ,
                    torch.tensor(edges[1]).unsqueeze(0)), dim=0).type(torch.long)

    #edges =data[trajectory]['cells'][ts]
    #print(edges.shape)
    #Get edge features
    u_i=torch.tensor(np.array(data[trajectory]['mesh_pos'][ts][edge_index[0]])) # source of the edges
    u_j=torch.tensor(np.array(data[trajectory]['mesh_pos'][ts][edge_index[1]])) # receivers of the edges
    u_ij=u_i-u_j
    u_ij_norm = torch.norm(u_ij,p=2,dim=1,keepdim=True)
    edge_attr = torch.cat((u_ij,u_ij_norm),dim=-1).type(torch.float)


    #Node outputs, for training (velocity)
    world_pos = torch.tensor(np.array(data[trajectory]['world_pos'][ts]))

    #Data needed for visualization code
    cells=torch.tensor(np.array(data[trajectory]['cells'][ts]))
    mesh_pos=torch.tensor(np.array(data[trajectory]['mesh_pos'][ts]))
    node_type = np.zeros_like(world_pos)

    data_list.append(Data( edge_index=edge_index, edge_attr=edge_attr,node_type=node_type,
                            cells=cells,mesh_pos=mesh_pos,world_pos=world_pos))

100%|██████████| 3999/3999 [00:00<00:00, 5516.06it/s]


In [10]:
print(data_list[3998]['cells'])

tensor([[0., 1., 2.],
        [0., 1., 3.]], dtype=torch.float64)


In [11]:
import os
torch.save(data_list,os.path.join('test_processed_set_2.pt'))
#torch.save(data_list,'./'+dataset_dir+'/test_processed_set.pt')

print("Done saving data!")


Done saving data!


In [20]:
import os
path = 'Data\flag_simple'
split = 'train'

tfrecord_path = os.path.join(path, split + ".tfrecord")
# index is generated by tfrecord2idx
index_path = os.path.join(path, split + ".idx")
tf_dataset = tf.data.TFRecordDataset(tfrecord_path, index_path, None)
# loader and iter(loader) have size 1000, which is the number of all training trajectories
loader = torch.utils.data.DataLoader(tf_dataset, batch_size=1)
# use list to make list from iterable so that the order of elements is ensured

ds_iterator = iter(loader)


In [22]:
ds_iterator.items()
print(dataset)

StopIteration: 

In [13]:
import torch
from torch.utils.data import DataLoader

dataset = torch.load('test_processed_set_2.pt')
loader = DataLoader(dataset, batch_size=2)
print(len(dataset))
print(dataset[0])
ds_iterator = iter(loader)
trajectory = next(ds_iterator)
trajectory.items()
#for key, value in trajectory_data.items():


3999
Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3])


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

In [ ]:
dataset = torch.load('meshgraphnets_miniset5traj_vis.pt')

FileNotFoundError: [Errno 2] No such file or directory: 'meshgraphnets_miniset5traj_vis.pt'

In [ ]:
print(dataset[100]['x'][-9:-1])

IndexError: list index out of range

In [ ]:
dataset = torch.load('test_processed_set_2.pt')
print(dataset)

[Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=[4, 3]), Data(edge_index=[2, 10], edge_attr=[10, 4], node_type=[4, 3], cells=[2, 3], mesh_pos=[4, 3], world_pos=

In [ ]:
data_list[0].x.shape[1:]

torch.Size([6])

In [ ]:
import tensorflow as tf
import os
import functools
import reading_utils

ds = tf.data.TFRecordDataset(['Data\flag_simple\train.tfrecord'])
ds = ds.map(functools.partial(
    reading_utils.parse_serialized_simulation_example, metadata='meta.json'))

TypeError: in user code:

    File "c:\Users\leixi\Downloads\meshgraphnets-main\data_creation\reading_utils.py", line 89, in parse_serialized_simulation_example  *
        position_shape = [metadata['sequence_length'] + 1, -1, metadata['dim']]

    TypeError: string indices must be integers, not 'str'


In [ ]:
import tensorflow as tf
import json

dataset = tf.data.TFRecordDataset("test.tfrecord")


In [ ]:
dataset = dataset.map('meta.json')